<a href="https://colab.research.google.com/github/Aynahd/Y24/blob/main/combine2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'german-traffic-sign-detection-benchmark-gtsdb:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F985376%2F1664166%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240508%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240508T082329Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D30615becf4d5d4e9043080da744bb794b8edd631b0cd4468346282ffd606586d8de30479cc6a599b4dbafbab4c26321a5fab78f1ab73045e5400ca0397598c7ceb823c9fd0142e66108c48d03ee7f04af56909ddcb3589fa420b3c0bf7ba5a67c92ad5436a475ec203d0af02417e7872259ead76c1cd2bab97e30e7d791838aefa0955040644bab4c9a084869df92e2d091e06c94eeddc8e55e676d9505c9a574b1d1998ca3397e695b3a1a155c0cdd08878fe9a5a2603b53e41c73cb0887ad88565c9377cee0592be8e7be71d33fd1d52bf4deeeaa8134316317b01c9eccd483a4c0dcca944d47e7c43dad462f2341079fb2ce63d215ab2621dbb3f3f026dd7,models1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3879927%2F6745249%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240508%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240508T082329Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D25023bf7f1adb7c19e5e41d0e8ab0f44d003f675695cbd5e828176c338df2eac9138a226d874392f3d44e439f66e837227aceb73e3e9e04351135f5f13674704ff90c4d4ad4817261419a989ab0f959d9eed4f3adfc35914df8ecc8dd1d05561d75213743192268d0c8adfafac71dd29afd3842d26c20d2c4d4d70793393235e2c9a402edab9b65ec993087118879b6ed4a35bb39202b3785b22dae9442ac6748ce5410b154c810a1a36ac2ed5048bd4d4c0dfc77af1e6002d3c70982c08ee3be45935cb7cf596887157e2cf3253c254b08655f5b102abec747e18648c63ebf28ba58ebabd7c6a750afab9c3f5ebe7bc5685100b18924f6af4063db4bfca3c52'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1729369023 bytes downloaded
Downloaded and uncompressed: german-traffic-sign-detection-benchmark-gtsdb
[==================================================] 18714046 bytes downloaded
Downloaded and uncompressed: models1
Data source import complete.


In [2]:
import torch
from PIL import Image
import shutil
import os
from random import choice
import numpy as np
import pandas as pd

In [3]:
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [4]:
%cd yolov5/

/content/yolov5


In [5]:
# ims = []
# for dirname, _, filenames in os.walk('/kaggle/input/german-traffic-sign-detection-benchmark-gtsdb/TestIJCNN2013/TestIJCNN2013Download'):
#     for filename in filenames:
#         if filename.endswith(".ppm"):
#             im = Image.open(os.path.join(dirname, filename))
#             ims.append(im)

In [6]:
ground_truth = {'boxes':[],'labels':[]}

In [7]:
f = open('/kaggle/input/german-traffic-sign-detection-benchmark-gtsdb/gt.txt')
dirname = "/kaggle/input/german-traffic-sign-detection-benchmark-gtsdb/TrainIJCNN2013/TrainIJCNN2013"
ims = []
y_true = []
boxs = []
labels = []
curname = ''
for x in f:
    datas = x.split(';')
    filename = datas[0]
    name = datas[0].split('.')[0]
    if curname != name:
        curname = name
        ground_truth['boxes'] = boxs
        ground_truth['labels'] = labels
        y_true.append(ground_truth)
        ground_truth = {'boxes':[],'labels':[]}
        boxs = []
        labels = []
        im = Image.open(os.path.join(dirname, filename))
        ims.append(im)
    boxs.append([int(i) for i in [datas[1],datas[2],datas[3],datas[4]]])
    labels.append(int(datas[5]))
ground_truth['boxes'] = boxs
ground_truth['labels'] = labels
y_true.append(ground_truth)
y_true.pop(0)
f.close()

In [8]:
print(len(y_true),len(ims))
print(y_true[505])

506 506
{'boxes': [[700, 454, 722, 476]], 'labels': [9]}


In [ ]:
import torch

# Model
model1 = torch.hub.load('Ultralytics/yolov5', 'custom','/kaggle/input/models1/best.pt', force_reload=True,trust_repo=True)


# Inference
results = model1(ims)

results.pandas()

Downloading: "https://github.com/Ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-5-8 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [ ]:
results1 = results.pandas().xyxy

In [ ]:
print(results1[50])

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
!pip install mapcalc

In [ ]:
import cv2
def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img
def equalize(img):
    img = cv2.equalizeHist(img)
    return img
def preprocess(img):
    img = grayscale(img)
    img = equalize(img)
    img = img/255
    return img
def normaly(img):
    img = np.asarray(img)
    img = cv2.resize(img, (32, 32))
    img = preprocess(img)
    #plt.imshow(img, cmap = plt.get_cmap('gray'))
    #plt.show()
    img = img.reshape(1, 32, 32, 1)
    return img
def relu(x):
    return x * (x > 0)

In [ ]:
import torchvision.transforms as transforms
data_transforms = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.Resize ((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

nclasses = 43
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 1, kernel_size=1)
        self.conv2 = nn.Conv2d(1, 29, kernel_size=5)
        self.maxpool2 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv3 = nn.Conv2d(29, 59, kernel_size=3)
        self.maxpool3 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv4 = nn.Conv2d(59, 74, kernel_size=3)
        self.maxpool4 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv2_drop = nn.Dropout2d()
        self.conv3_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(1184, 300)
        self.fc2 = nn.Linear(300, nclasses)
        self.conv0_bn = nn.BatchNorm2d(3)
        self.conv1_bn = nn.BatchNorm2d(1)
        self.conv2_bn = nn.BatchNorm2d(29)
        self.conv3_bn = nn.BatchNorm2d(59)
        self.conv4_bn = nn.BatchNorm2d(74)
        self.dense1_bn = nn.BatchNorm1d(300)
    def forward(self, x):
        x =  F.relu(self.conv1_bn(self.conv1(self.conv0_bn(x))))
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.relu(self.conv3_bn(self.conv3( self.maxpool2(x))))
        x = F.relu(self.conv4_bn(self.conv4( self.maxpool3(x))))
        x = self.maxpool4(x)
        x = x.view(-1, 1184)
        x = F.relu(self.fc1(x))
        x = self.dense1_bn(x)
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.dropout(x, training=self.training)
        return F.log_softmax(x)

In [ ]:
model2 = Net()
model2.load_state_dict(torch.load('/kaggle/input/models1/model_74.pth'))
model2.eval()

In [ ]:
import requests
from PIL import Image
url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS1C3nUKC8VUu5nWTZao2M367MUPVLsTMzYVA&usqp=CAU'
r = requests.get(url, stream=True)
img = Image.open(r.raw)
plt.imshow(img, cmap=plt.get_cmap('gray'))
numpy_array = np.array(img)
ttt = Image.fromarray(numpy_array)

foo = data_transforms(ttt)
print(foo)
# bar = foo(img)
foo = foo.reshape(1,3,48,48)

In [ ]:
print(foo[0].shape)

In [ ]:

testing =model2(foo)

In [ ]:
print(int(torch.argmax(testing[0])))
print(torch.exp(testing))

In [ ]:
print(y_true[504])

In [ ]:
from mapcalc import calculate_map, calculate_map_range
map50s = []
crops = []
import cv2
for i in range(len(results1)):
    result_dict = {'boxes':[],'labels':[],'scores':[]}
    boxs = []
    labels = []
    scores = []
    for index,row in results1[i].astype({"xmin":'int', "ymin":'int', "xmax":'int', "ymax":'int'}).iterrows():
        ima = np.array(ims[i])
    #print(row)
        xmin = relu(row['xmin'] - 5)
        ymin = relu(row['ymin'] - 5)
        xmax = relu(row['xmax'] + 5)
        ymax = relu(row['ymax'] + 5)
    #cv2.rectangle(ima, (xmin, ymin), (xmax, ymax), (255,0,0), 4)
        cropped_image = ima[ymin:ymax, xmin:xmax]
        #crops.append(cropped_image)
        #plt.imshow(cropped_image)
        #plt.show()
        try:
            im = Image.fromarray(cropped_image)
            im = data_transforms(im)
            im = im.reshape(1,3,48,48)
            y_pred1 = model2(im)
            y_pred1 = torch.exp(y_pred1)
            la = int(torch.argmax(y_pred1[0]))
            boxs.append([row['xmin'],row['ymin'],row['xmax'],row['ymax']])
            labels.append(la)
            scores.append(y_pred1[0][la])
        except Exception as error:
            print(i,error)

        #print("predicted sign: "+ str(la))
    result_dict['boxes'] = boxs
    result_dict['labels'] = labels
    result_dict['scores'] = scores
    ground_truth = y_true[i]
    map50 = calculate_map(ground_truth, result_dict, 0.5)
    if map50 < 0.8:
        print(ground_truth)
        print(result_dict)
        print(map50)
    map50s.append(map50)

In [ ]:
print(map50s)
def Average(lst):
    return sum(lst) / len(lst)

# Driver Code
average = Average(map50s)

# Printing average of the list
print("Average of the list =", round(average, 2))

In [ ]:
cropped_image = ima[Y:Y+H, X:X+W]
print([X,Y,W,H])
plt.imshow(cropped_image)

In [ ]:
import requests
from PIL import Image
url = 'https://thumbs.dreamstime.com/t/road-signs-main-road-sign-blue-background-road-signs-main-road-sign-blue-background-109436823.jpg'
r = requests.get(url, stream=True)
img = Image.open(r.raw)
plt.imshow(img, cmap=plt.get_cmap('gray'))
#im = normaly(img)
img = np.asarray(img)
img = cv2.resize(img, (32, 32))
img = preprocess(img)
plt.imshow(img, cmap = plt.get_cmap('gray'))
print(img.shape)
img = img.reshape(1, 32, 32, 1)
y_pred1 = model2.predict(img)
la = np.argmax(y_pred1,axis=1)
print("predicted sign: "+ str(la))

In [ ]:
from mapcalc import calculate_map, calculate_map_range
ground_truth = {
    'boxes':
        [[60., 80., 66., 92.],
         [70., 87., 81., 94.],
         [8., 34., 10., 36.]],

    'labels':
        [2, 3, 4]}

result_dict = {
    'boxes':
        [[60., 80., 66., 92.],
         [59., 94., 68., 97.],
         [70., 87., 81., 94.],
         [8., 34., 10., 36.]],

    'labels':
        [2, 3, 3, 4],

    'scores':
        [0.99056727, 0.98965424, 0.93990153, 0.9157755]}

# calculates the mAP for an IOU threshold of 0.5
print(calculate_map(ground_truth, result_dict, 0.5))

# calculates the mAP average for the IOU thresholds 0.05, 0.1, 0.15, ..., 0.90, 0.95.
print(calculate_map_range(ground_truth, result_dict, 0.05, 0.95, 0.05))

In [ ]:
from mapcalc import calculate_map, calculate_map_range
ground_truth = {
    'boxes':
        [[60., 80., 66., 92.],
         [70., 87., 81., 94.],
         [8., 34., 10., 36.]],

    'labels':
        [2, 3, 4]}

result_dict = {
    'boxes':
        [[70., 87., 81., 94.],
         [60., 80., 66., 92.],
         [8., 34., 10., 36.]],

    'labels':
        [3, 2, 4],

    'scores':
        [0.99056727, 0.98965424, 0.93990153]}

# calculates the mAP for an IOU threshold of 0.5
print(calculate_map(ground_truth, result_dict, 0.5))

# calculates the mAP average for the IOU thresholds 0.05, 0.1, 0.15, ..., 0.90, 0.95.
print(calculate_map_range(ground_truth, result_dict, 0.05, 0.95, 0.05))